In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm

In [2]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-"
suffix = "/raw/main/results/output/output_df.csv"
suffix_squadid = "/resolve/main/results/output/output_df.csv"

In [3]:
def take_dataframe(url): 
    
    response = requests.get(url)
    
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data, index_col=0)
        df = df[['Prediction Answer', 'Rec. Pred Answer', 'Gold Answer', 'Properties']]
        df.fillna('', inplace=True)
    
    else:
        print("Failed to download CSV")
    
    return df

In [4]:
data = "idkmrc"
url = f"{prefix}indonli_mnli_{data}-nli-{data}-TQ2-TS4-MS3-VA3-TH0.25{suffix}"

In [5]:
url

'https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-indonli_mnli_idkmrc-nli-idkmrc-TQ2-TS4-MS3-VA3-TH0.25/raw/main/results/output/output_df.csv'

In [8]:
x = take_dataframe(url)
x

,Prediction Answer,Rec. Pred Answer,Gold Answer,Properties
0,1959,"['1959', '1971', '']",1959,Answer saved from variation 3 with founded index
1,sirkuit terpadu (IC) yang mulai dikembangkan p...,"['', '', 'sirkuit terpadu (IC) yang mulai dike...",,Answer saved from variation 3 with founded index
2,1959,"['', '1959', 'GAN yang sedikit ngawur di bidan...",,Answer saved from variation 3 with founded index
3,1959,"['', '1959', '1971']",,Answer saved from variation 3 with founded index
4,komponen dari sebuah nama pribadi yang berdasa...,['komponen dari sebuah nama pribadi yang berda...,Komomene sebuah nama yang berdasarkan pada nam...,Answer saved from variation 3 with founded index
...,...,...,...,...
839,"bereproduksi, dan hanya memakan getah pohon hi...","['', 'bereproduksi, dan hanya memakan getah po...",,Answer saved from variation 3 with founded index
840,131.3 mil persegi,"['131.3 mil persegi', '', 'Menurut Biro Sensus...","Menurut Biro Sensus Amerika Serikat, kota ini ...",Answer saved from variation 3 with founded index
841,"Menurut Biro Sensus Amerika Serikat, kota ini ...","['', 'Menurut Biro Sensus Amerika Serikat, kot...",,Answer saved from variation 3 with founded index
842,bisnis ayahnya terkena dampak depresi pertenga...,['bisnis ayahnya terkena dampak depresi perten...,Ogilvy tidak bisa kuliah di Fettes atau Oxford...,Answer saved from variation 3 with founded index


In [14]:
eval(x['Rec. Pred Answer'][0])[0]

'1959'

In [30]:
def take_accepted_and_rejected_qa_with_nli(df):
    
    right_pred_model_qa_and_accepted_by_nli_ctr = 0
    right_pred_model_qa_and_rejected_by_nli_ctr = 0
    wrong_pred_model_qa_and_accepted_by_nli_ctr = 0
    wrong_pred_model_qa_and_rejected_by_nli_ctr = 0
    
    for i in tqdm(range(len(df))):
        
        gold_answer = df['Gold Answer'][i]
        qa_generated_answer = eval(x['Rec. Pred Answer'][i])[0]
        nli_validated_answer = df['Prediction Answer'][i]
    
        # Jawaban dari model qa yg benar & di-accept sama model NLI
        right_pred_model_qa_and_accepted_by_nli = (gold_answer == qa_generated_answer) and \
            (qa_generated_answer == nli_validated_answer)

        # Jawaban dari model qa yg benar & di-reject sama model NLI
        right_pred_model_qa_and_rejected_by_nli = (gold_answer == qa_generated_answer) and \
            (qa_generated_answer != nli_validated_answer)

        # Jawaban dari model qa yg salah & di-accept sama model NLI
        wrong_pred_model_qa_and_accepted_by_nli = (gold_answer != qa_generated_answer) and \
            (qa_generated_answer == nli_validated_answer)

        # Jawaban dari model qa yg salah & di-reject sama model NLI
        wrong_pred_model_qa_and_rejected_by_nli = (gold_answer != qa_generated_answer) and \
            (qa_generated_answer != nli_validated_answer)
        
        if right_pred_model_qa_and_accepted_by_nli:
            right_pred_model_qa_and_accepted_by_nli_ctr += 1
        elif right_pred_model_qa_and_rejected_by_nli:
            right_pred_model_qa_and_rejected_by_nli_ctr += 1
        elif wrong_pred_model_qa_and_accepted_by_nli:
            wrong_pred_model_qa_and_accepted_by_nli_ctr += 1
        elif wrong_pred_model_qa_and_rejected_by_nli:
            wrong_pred_model_qa_and_rejected_by_nli_ctr += 1
    
    return right_pred_model_qa_and_accepted_by_nli_ctr, right_pred_model_qa_and_rejected_by_nli_ctr, \
            wrong_pred_model_qa_and_accepted_by_nli_ctr, wrong_pred_model_qa_and_rejected_by_nli_ctr

In [31]:
a, b, c, d = take_accepted_and_rejected_qa_with_nli(x)

100%|██████████████████████████████████████████████████████████████████████████████| 844/844 [00:00<00:00, 30755.53it/s]


In [32]:
a+b+c+d

844

In [ ]:
# Jawaban dari model qa yg benar & di-accept sama model NLI
right_pred_model_qa_and_accepted_by_nli = (Gold Answer == Rec.Pred Answer[0]) and (Rec.Pred Answer[0] == Prediction Answer)

# Jawaban dari model qa yg benar & di-reject sama model NLI
right_pred_model_qa_and_rejected_by_nli = (Gold Answer == Rec.Pred Answer[0]) and (Rec.Pred Answer[0] != Prediction Answer)

# Jawaban dari model qa yg salah & di-accept sama model NLI
wrong_pred_model_qa_and_accepted_by_nli = (Gold Answer != Rec.Pred Answer[0]) and (Rec.Pred Answer[0] == Prediction Answer)

# Jawaban dari model qa yg salah & di-reject sama model NLI
wrong_pred_model_qa_and_rejected_by_nli = (Gold Answer != Rec.Pred Answer[0]) and (Rec.Pred Answer[0] != Prediction Answer)

In [6]:
right_pred_model_qa = (Gold Answer == Prediction Answer) and (Prediction Answer == Rec.Pred Answer[0])
wrong_pred_model_qa = (Gold Answer != Prediction Answer) and (Prediction Answer == Rec.Pred Answer[0])

right_pred_model_nli = (Gold Answer == Prediction Answer) and (Prediction Answer != Rec.Pred Answer[0])
wrong_pred_model_nli = (Gold Answer != Prediction Answer) and (Prediction Answer != Rec.Pred Answer[0])

SyntaxError: invalid syntax (199361488.py, line 1)